nanoGPT
========
Here I'm following a [tutorial from Andrej Karpathy](https://youtu.be/kCc8FmEb1nY)

We'll be using a transformer NN to mimic a very primitive version chatGPT

deps
-------
pip install tiktoken  
pip install torch  

abrev
--------
GPT: Generatively Pretrained Transformer  


In [32]:
# get a dataset
import urllib
text = urllib.request.urlopen('https://github.com/karpathy/char-rnn/raw/master/data/tinyshakespeare/input.txt').read().decode('ascii')
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [12]:
chars = sorted(set(text))
vocab_size = len(chars)
print(''.join(char), vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz 65


In [29]:
# encode the input into a computer friendly format
# here we are tokenizing character by character, in real life chatGPT uses a byte-pair encoding (BPE),
# it's a sub-word units, which means they don't use entire words as token, only chunks of binary

# in real life we'd use something like
# import tiktoken
# enc = tiktoken.get_encoding('gpt2')
# enc.encode('I for one welcome our new AI overlords')
# enc.decode([40, 329, 530, 7062, 674, 649, 9552, 12893, 3669])

# for our toy we use some manual encoding
stoi = { ch: i for i, ch in enumerate(chars) }
itos = { i: ch for i, ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

# we have a good candidate for quickcheck here
s = 'string generated by quickcheck from our vocab'
assert s == decode(encode(s)), f'{s} did not encore/decode correctly'

print(encode('I for one welcome our new AI overlords'))
print(decode([21, 1, 44, 53, 56, 1, 53, 52, 43, 1, 61, 43, 50, 41, 53, 51, 43, 1, 53, 59, 56, 1, 52, 43, 61, 1, 13, 21, 1, 53, 60, 43, 56, 50, 53, 56, 42, 57]))

[21, 1, 44, 53, 56, 1, 53, 52, 43, 1, 61, 43, 50, 41, 53, 51, 43, 1, 53, 59, 56, 1, 52, 43, 61, 1, 13, 21, 1, 53, 60, 43, 56, 50, 53, 56, 42, 57]
I for one welcome our new AI overlords


In [31]:
# encode the dataset
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [34]:
# split the data into training set and validation set
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [65]:
# chunk the dataset into smaller pieces for training
# in practice we will train in all sizes < chunk
# e.g. for a chunk size of 8 we will train on chunks of size 1 to 8
block_size = 8
# to speedup training we run several examples at a time
batch_size = 32

def get_batch(dataset):
    data = train_data if dataset == 'train' else val_data
    ix = torch.randint(len(data) - block_size - 1, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print(xb)
print(yb)

tensor([[59, 58,  1, 45, 43, 52, 58, 50],
        [22, 59, 52, 53,  7, 50, 47, 49],
        [39, 58, 46,  1, 53, 52, 50, 63],
        [47, 52, 42, 57,  1, 17, 42, 61],
        [ 1, 57, 46, 39, 50, 50,  1, 40],
        [56, 42,  6,  1, 21,  1, 58, 46],
        [21, 27, 10,  0, 27,  6,  1, 60],
        [57,  1, 39, 52, 42,  1, 51, 53],
        [49,  8,  0,  0, 15, 50, 53, 61],
        [43, 58,  8,  0, 21,  5, 50, 42],
        [ 1, 39,  1, 46, 53, 50, 63,  1],
        [43,  1, 39, 41, 41, 53, 51, 54],
        [53, 59, 56,  1, 57, 61, 53, 56],
        [ 5, 42,  1, 46, 43, 39, 60, 43],
        [ 0, 46, 59, 52, 58,  1, 58, 46],
        [58,  1, 58, 46, 43, 63,  1, 63],
        [ 1, 42, 53,  1, 52, 43, 43, 42],
        [43, 39, 56,  1, 40, 59, 58,  1],
        [42,  1, 39, 56, 51,  5, 42, 12],
        [10,  0, 28, 50, 39, 52, 58, 39],
        [58, 41, 46,  0, 32, 46, 39, 58],
        [ 1, 51, 39, 48, 43, 57, 58, 63],
        [43, 57, 43, 43, 41, 46,  1, 63],
        [63,  1, 58, 53, 53,  8,  

In [59]:
import torch.nn as nn
from torch.nn import functional as F

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):
        # Batch (aka batch_size) * Time (aka block_size) * Channel (aka vocab_size)
        logits = self.token_embedding_table(idx) # (B, T, C)
        
        if targets == None:
            loss = None
        else:
            # massage the vector shapes to match the cross_entropy function
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            # how well are we prediting targets based on logits
            # (how well are we predicting next character basec on current character)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # TODO: MAGIC HAPPENING HERE
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx
    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

# generate a 100 token string starting from the first character of our alphabet
# in this case starting with a '\n'
# at this point it's random based on the random initial weights of the NN
idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.7503, grad_fn=<NllLossBackward0>)

t JXXGzCJ. Az3dsQcP$SjeJVk?&YqMGdE!J.K?MbZU:.L,oEbz3SB.h$ZFCp-DbOdD XSlI-tH:e-LahLNVGh-a.eJ3QRiSuaaB


In [60]:
# choose an pytorch optimizer
# ---------------------------
# Adam is fast but overfit
# SGD is slow but has good results
# in some case RMSProp or SWA are better
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [83]:
# train the model
for steps in range(10000):
    xb, yx = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

3.1933352947235107


In [88]:
# doesn't look that much better at this point, but it should be a bit less random
print(decode(m.generate(torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


otPdytwd b wwerceashbl.tdii
 tbl a
    nhc
a'om,tets .es
 etI,d
yerur,eieet.trdhueuteenk tn hoslfl 

